In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
PROJECT_FOLDER = "/content/drive/MyDrive/HealthMateAI_Project"

if not os.path.exists(PROJECT_FOLDER):
    os.makedirs(PROJECT_FOLDER)
    print(f"✅ Permanent Project Folder Created: {PROJECT_FOLDER}")

Mounted at /content/drive
✅ Permanent Project Folder Created: /content/drive/MyDrive/HealthMateAI_Project


In [ ]:
%%writefile /content/drive/MyDrive/HealthMateAI_Project/chatbot_logic.py
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="YOUR API KEY")

def get_health_response(user_query, chat_history):
    # Updated System Prompt to include your biography
    system_prompt = (
        "You are 'HealthMate AI', a professional and empathetic medical assistant. "
        "You were created by Tahir, a talented AI Developer and Intern. "
        "If anyone asks who made you or created you, proudly mention Tahir and his work in AI. "
        "Always provide science-based health information and end with a medical disclaimer."
    )

    emergencies = ["heart attack", "stroke", "chest pain", "suicide"]
    if any(k in user_query.lower() for k in emergencies):
        return "🚨 EMERGENCY: Please call 911 immediately."

    messages = [{"role": "system", "content": system_prompt}] + chat_history + [{"role": "user", "content": user_query}]

    try:
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.2-3B-Instruct",
            messages=messages,
            max_tokens=500
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"Service Error: {str(e)}"

Overwriting /content/drive/MyDrive/HealthMateAI_Project/chatbot_logic.py


In [6]:
%%writefile /content/drive/MyDrive/HealthMateAI_Project/app.py
import streamlit as st
import sys
import os

# Link to our project logic
sys.path.append("/content/drive/MyDrive/HealthMateAI_Project")
from chatbot_logic import get_health_response

# 1. Page Configuration & Theme
st.set_page_config(page_title="HealthMate AI", page_icon="🏥", layout="wide")

# 2. Professional Sidebar (Branding Tahir)
with st.sidebar:
    st.image("https://cdn-icons-png.flaticon.com/512/387/387561.png", width=100)
    st.title("Developer Info")
    st.markdown("---")
    st.write("**Created by:** Tahir")
    st.write("**Role:** AI Developer Intern")
    st.write("**Project:** HealthMate AI v1.0")
    st.markdown("---")
    st.info("This AI provides general health information. For emergencies, please contact local medical services.")

    if st.button("🗑️ Clear Conversation"):
        st.session_state.chat_history = []
        st.rerun()

# 3. Main Interface
st.title("🏥 HealthMate AI Assistant")
st.markdown("#### *Your Intelligent Path to Wellness*")
st.caption(f"Developed by Tahir | Powered by Llama-3.2")

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Display conversation
for msg in st.session_state.chat_history:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# 4. Chat Input
if user_input := st.chat_input("How can I help you today?"):
    with st.chat_message("user"):
        st.markdown(user_input)

    # AI logic call
    response = get_health_response(user_input, st.session_state.chat_history)

    with st.chat_message("assistant"):
        st.markdown(response)

    st.session_state.chat_history.append({"role": "user", "content": user_input})
    st.session_state.chat_history.append({"role": "assistant", "content": response})

Overwriting /content/drive/MyDrive/HealthMateAI_Project/app.py


In [ ]:
import os
import subprocess
import sys
import time

# 1. Install/Verify Tools (Needed every time you restart Colab)
print("🏗️ Preparing environment...")
!pip install -q streamlit huggingface_hub
if not os.path.exists("./cloudflared"):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
    !chmod +x cloudflared

# 2. Start Streamlit from your Permanent Drive Path
APP_PATH = "/content/drive/MyDrive/HealthMateAI_Project/app.py"
print(f"🔋 Launching {APP_PATH}...")

# Headless=true prevents Colab from trying to open a browser window internally
subprocess.Popen([sys.executable, "-m", "streamlit", "run", APP_PATH,
                  "--server.port=8501", "--server.headless=true"])

# 3. Give the server time to warm up
time.sleep(10)

# 4. Open the Cloudflare Tunnel
print("🔗 Your Public Link is generating! Look for the 'trycloudflare.com' link below: \n")
!./cloudflared tunnel --url http://localhost:8501 --protocol http2

🏗️ Preparing environment...
🔋 Launching /content/drive/MyDrive/HealthMateAI_Project/app.py...
🔗 Your Public Link is generating! Look for the 'trycloudflare.com' link below: 

2026-02-14T17:32:06Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-14T17:32:06Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-02-14T17:32:11Z INF +--------------------------------------------------------------------------------------------+
2026-02-14T17:32:11Z INF |